## setup

In [1]:
import pandas as pd
import cv2
import os

import time

import requests
import json
import urllib
from bs4 import BeautifulSoup

# show all dataframe
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_columns', None)  

## Basic Requests Checkout

### MET


MET guideline here:<br>
https://metmuseum.github.io/

In [68]:
# checkout all departments
url = "https://collectionapi.metmuseum.org/public/collection/v1/departments"
r = requests.get(url)
r

<Response [200]>

In [69]:
r.text

'{"departments":[{"departmentId":1,"displayName":"American Decorative Arts"},{"departmentId":3,"displayName":"Ancient Near Eastern Art"},{"departmentId":4,"displayName":"Arms and Armor"},{"departmentId":5,"displayName":"Arts of Africa, Oceania, and the Americas"},{"departmentId":6,"displayName":"Asian Art"},{"departmentId":7,"displayName":"The Cloisters"},{"departmentId":8,"displayName":"The Costume Institute"},{"departmentId":9,"displayName":"Drawings and Prints"},{"departmentId":10,"displayName":"Egyptian Art"},{"departmentId":11,"displayName":"European Paintings"},{"departmentId":12,"displayName":"European Sculpture and Decorative Arts"},{"departmentId":13,"displayName":"Greek and Roman Art"},{"departmentId":14,"displayName":"Islamic Art"},{"departmentId":15,"displayName":"The Robert Lehman Collection"},{"departmentId":16,"displayName":"The Libraries"},{"departmentId":17,"displayName":"Medieval Art"},{"departmentId":18,"displayName":"Musical Instruments"},{"departmentId":19,"display

In [70]:
# search a specific department
url = "https://collectionapi.metmuseum.org/public/collection/v1/objects?departmentIds=8"
r = requests.get(url)
r

<Response [200]>

In [71]:
costume_objectsIDs = r.json()['objectIDs']
len(costume_objectsIDs)

31550

In [6]:
# Search by a topic
url = "https://collectionapi.metmuseum.org/public/collection/v1/search?q=robe"
r = requests.get(url)
r

<Response [200]>

In [7]:
r.json().keys()

dict_keys(['total', 'objectIDs'])

### V&A

guidelines here:<br>
https://api.vam.ac.uk/docs#/Archives/archive_search_v2_archives_search_get<br>
https://developers.vam.ac.uk/guide/v2/images/introduction.html#images-top

In [2]:
# checkout a topic
url = "https://api.vam.ac.uk/v2/objects/search?q_object_name=robe"
r = requests.get(url)
r

<Response [200]>

In [3]:
r.json().keys()

dict_keys(['info', 'records', 'clusters'])

In [4]:
r.json()

{'info': {'version': '2.0',
  'record_count': 6178,
  'record_count_exact': True,
  'parameters': {},
  'page_size': 15,
  'pages': 412,
  'page': 1,
  'image_count': 6261},
 'records': [{'systemNumber': 'O1227158',
   'accessionNumber': '1987(IS)',
   'objectType': 'Robe',
   '_currentLocation': {'id': 'THES50151',
    'displayName': 'In store',
    'type': 'storage',
    'site': 'BH',
    'onDisplay': False,
    'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}},
   '_primaryTitle': 'Robe',
   '_primaryMaker': {'name': 'Unknown', 'association': ''},
   '_primaryImageId': '2017KB8846',
   '_primaryDate': 'ca. 1873',
   '_primaryPlace': 'Shache',
   '_warningTypes': [],
   '_images': {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2017KB8846/full/!100,100/0/default.jpg',
    '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2017KB8846/',
    '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O1227158/manifest.json',
    'imageResolutio

In [7]:
r.json()['records']

[{'systemNumber': 'O1227158',
  'accessionNumber': '1987(IS)',
  'objectType': 'Robe',
  '_currentLocation': {'id': 'THES50151',
   'displayName': 'In store',
   'type': 'storage',
   'site': 'BH',
   'onDisplay': False,
   'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}},
  '_primaryTitle': 'Robe',
  '_primaryMaker': {'name': 'Unknown', 'association': ''},
  '_primaryImageId': '2017KB8846',
  '_primaryDate': 'ca. 1873',
  '_primaryPlace': 'Shache',
  '_warningTypes': [],
  '_images': {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2017KB8846/full/!100,100/0/default.jpg',
   '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2017KB8846/',
   '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O1227158/manifest.json',
   'imageResolution': 'high'}},
 {'systemNumber': 'O363932',
  'accessionNumber': 'T.66-1913',
  'objectType': 'Robe',
  '_currentLocation': {'id': 'THES50151',
   'displayName': 'In store',
   'type': 'storage',
   'site'

In [11]:
y = pd.DataFrame.from_dict(r.json()['records'])
y

,systemNumber,accessionNumber,objectType,_currentLocation,_primaryTitle,_primaryMaker,_primaryImageId,_primaryDate,_primaryPlace,_warningTypes,_images
0,O1227158,1987(IS),Robe,"{'id': 'THES50151', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,"{'name': 'Unknown', 'association': ''}",2017KB8846,ca. 1873,Shache,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2017KB8846/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2017KB8846/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O1227158/manifest.json', 'imageResolution': 'high'}"
1,O363932,T.66-1913,Robe,"{'id': 'THES50151', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,{},2006AN8249,1800s,Republic of Liberia,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2006AN8249/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2006AN8249/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O363932/manifest.json', 'imageResolution': 'high'}"
2,O143220,266-1890,Robe,"{'id': 'THES50151', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,"{'name': 'Unknown', 'association': ''}",2006AN8244,1850-1875,Egypt,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2006AN8244/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2006AN8244/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O143220/manifest.json', 'imageResolution': 'high'}"
3,O363080,CIRC.182-1962,Robe,"{'id': 'THES50151', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,"{'name': 'Unknown', 'association': ''}",2019LP1268,1900-1940,Africa,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2019LP1268/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2019LP1268/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O363080/manifest.json', 'imageResolution': 'high'}"
4,O350946,1166-1903,Robe,"{'id': 'THES49199', 'displayName': 'In store', 'type': 'storage', 'site': 'VA', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,{},2019LR3422,1850-1900,Nigeria,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2019LR3422/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2019LR3422/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O350946/manifest.json', 'imageResolution': 'high'}"
5,O91369,1110-1898,Robe,"{'id': 'THES49199', 'displayName': 'In store', 'type': 'storage', 'site': 'VA', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe,"{'name': 'Unknown', 'association': ''}",2019LR3426,1880s,Mali,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2019LR3426/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2019LR3426/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O91369/manifest.json', 'imageResolution': 'high'}"
6,O363414,13-1900,Robe and trousers,"{'id': 'THES50151', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}",Robe and trousers,{},2019LP1263,1870-1899,Nigeria,[],"{'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2019LP1263/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2019LP1263/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O363414/manifest.json', 'imageResolution': 'h

## prototype for dataframes concat function

In [9]:
# checkout a specific object by object id
url = "https://collectionapi.metmuseum.org/public/collection/v1/objects/53713"
r = requests.get(url)
r

<Response [200]>

In [11]:
# then use the below code to get one object info
x = pd.DataFrame.from_dict(r.json(), orient='index').T
x

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,objectName,title,culture,period,dynasty,reign,portfolio,artistRole,artistPrefix,artistDisplayName,artistDisplayBio,artistSuffix,artistAlphaSort,artistNationality,artistBeginDate,artistEndDate,artistGender,artistWikidata_URL,artistULAN_URL,objectDate,objectBeginDate,objectEndDate,medium,dimensions,measurements,creditLine,geographyType,city,state,county,country,region,subregion,locale,locus,excavation,river,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,53713,False,32.30.10,1932,True,https://images.metmuseum.org/CRDImages/as/original/DP241377.jpg,https://images.metmuseum.org/CRDImages/as/web-large/DP241377.jpg,"[https://images.metmuseum.org/CRDImages/as/original/DP241381.jpg, https://images.metmuseum.org/CRDImages/as/original/DP241382.jpg]",None,Asian Art,Robe,Theatrical Robe for a Male Role,China,Qing dynasty (1644–1911),,,,,,,,,,,,,,,,second half of the18th century,1750,1799,Silk florentine stitch embroidery on silk gauze,Overall: 55 3/8 x 89 1/4 in. (140.7 x 226.7 cm),"[{'elementName': 'Overall', 'elementDescription': None, 'elementMeasurements': {'Height': 140.6528, 'Width': 226.6955}}]","Purchase, Joseph Pulitzer Bequest, 1932",,,,,,,,,,,,Costumes-Embroidered,,,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/53713,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.getty.edu/page/aat/300310335', 'Wikidata_URL': 'https://www.wikidata.org/wiki/Q28425'}]",,False,


In [81]:
url = "https://collectionapi.metmuseum.org/public/collection/v1/objects/79667"
r = requests.get(url)
r
y = pd.DataFrame.from_dict(r.json(), orient='index').T
y

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,79667,False,"1997.478.8a, b",1997,False,,,[],"[{'constituentID': 161959, 'role': 'Designer',...",Costume Institute,...,,,,2022-10-05T04:56:22.043Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,,False,


In [82]:
pd.concat([x,y])

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,53713,False,32.30.10,1932,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[https://images.metmuseum.org/CRDImages/as/ori...,None,Asian Art,...,Costumes-Embroidered,,,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.get...",,False,
0,79667,False,"1997.478.8a, b",1997,False,,,[],"[{'constituentID': 161959, 'role': 'Designer',...",Costume Institute,...,,,,2022-10-05T04:56:22.043Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,,False,


## scrape: The Costume Institute Department

In [83]:
type(costume_objectsIDs) # list of objects ID in the costume department created earlier

list

In [41]:
urlstring = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

for id in costume_objectsIDs:
    thisurl = thisurl = urlstring + str(id)
    r = requests.get(thisurl)
    thisdf = pd.DataFrame.from_dict(r.json(), orient='index').T

    df = pd.concat([df,thisdf])

# took 64m53s

In [88]:
df.to_csv(r'scrapped_data/MET_costume_dept.csv', index = False) 

In [89]:
df = pd.read_csv('scrapped_data/MET_costume_dept.csv') 

## def function for scrapping

### function: get_objectid_list from a certain topic

In [6]:
# define a function to get the object id list
# of a certain topic
def get_objectid_list(keyword):
    root = "https://collectionapi.metmuseum.org/public/collection/v1/search?q="
    target_url = root + keyword
    r = requests.get(target_url)
    idlist = r.json()['objectIDs']
    print(f'there are {len(idlist)} object(s) about {keyword}')

    return idlist

### function: scrapping

In [7]:
def scrapping(keyword):
   start = time.time()

   target_columns = ['objectID', 'isHighlight', 'accessionNumber', 'accessionYear',
       'isPublicDomain', 'primaryImage', 'primaryImageSmall',
       'additionalImages', 'constituents', 'department', 'objectName', 'title',
       'culture', 'period', 'dynasty', 'reign', 'portfolio', 'artistRole',
       'artistPrefix', 'artistDisplayName', 'artistDisplayBio', 'artistSuffix',
       'artistAlphaSort', 'artistNationality', 'artistBeginDate',
       'artistEndDate', 'artistGender', 'artistWikidata_URL', 'artistULAN_URL',
       'objectDate', 'objectBeginDate', 'objectEndDate', 'medium',
       'dimensions', 'measurements', 'creditLine', 'geographyType', 'city',
       'state', 'county', 'country', 'region', 'subregion', 'locale', 'locus',
       'excavation', 'river', 'classification', 'rightsAndReproduction',
       'linkResource', 'metadataDate', 'repository', 'objectURL', 'tags',
       'objectWikidata_URL', 'isTimelineWork', 'GalleryNumber']
       
       
   # initiate an empty df
   df = pd.DataFrame(columns=target_columns)

   idlist = get_objectid_list(keyword)

   urlstring = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

   for id in idlist:
      thisurl = thisurl = urlstring + str(id)
      r = requests.get(thisurl)
      thisdf = pd.DataFrame.from_dict(r.json(), orient='index').T
      df = pd.concat([df,thisdf])
   
   end = time.time()
   print(f'{end-start} secs slipped...')
   
   return df



### function: scrapping using an existing list

In [8]:
def scrapping_by_idlist(idlist):
   start = time.time()

   target_columns = ['objectID', 'isHighlight', 'accessionNumber', 'accessionYear',
       'isPublicDomain', 'primaryImage', 'primaryImageSmall',
       'additionalImages', 'constituents', 'department', 'objectName', 'title',
       'culture', 'period', 'dynasty', 'reign', 'portfolio', 'artistRole',
       'artistPrefix', 'artistDisplayName', 'artistDisplayBio', 'artistSuffix',
       'artistAlphaSort', 'artistNationality', 'artistBeginDate',
       'artistEndDate', 'artistGender', 'artistWikidata_URL', 'artistULAN_URL',
       'objectDate', 'objectBeginDate', 'objectEndDate', 'medium',
       'dimensions', 'measurements', 'creditLine', 'geographyType', 'city',
       'state', 'county', 'country', 'region', 'subregion', 'locale', 'locus',
       'excavation', 'river', 'classification', 'rightsAndReproduction',
       'linkResource', 'metadataDate', 'repository', 'objectURL', 'tags',
       'objectWikidata_URL', 'isTimelineWork', 'GalleryNumber']
       
       
   # initiate an empty df
   df = pd.DataFrame(columns=target_columns)

   urlstring = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

   for id in idlist:
      thisurl = thisurl = urlstring + str(id)
      r = requests.get(thisurl)
      thisdf = pd.DataFrame.from_dict(r.json(), orient='index').T
      df = pd.concat([df,thisdf])
   
   end = time.time()
   print(f'{end-start} secs slipped...')
   
   return df

### function: download_img

In [9]:
def download_img(df,dir='testing/'):
    start = time.time()
    print(f'{df.shape[0]} images will be downloaded.')

    name = df['objectID']
    url = df['primaryImage']

    name = list(name)
    url = list(url)

    error_id = []
    error_n = 0

    for n,u in zip(name,url):
        filename = dir + str(n) + '.jpg'
        try:
            z = urllib.request.urlopen(u)
            output = open(filename, 'wb')
            output.write(z.read())
            output.close()
        except:
            error_id.append(n)
            print(f'error id {n}')
            error_n += 1
            print(f'{error_n} errors in total')
            continue


    end = time.time()
    print(f'{end - start} secs slipped...')

## scrape: robe

this is a test batch, for detecting filtering issues

found issues:
- unrelated topics: may be fixed by filtering classification, single word topic
- black & white

### download dataframe

In [125]:
df_robe = scrapping('robe')

there are 2470 object(s) about robe
685.2680084705353 secs slipped...


In [126]:
df_robe

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message
0,53715,False,43.119,1943,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],None,Asian Art,...,,,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.get...",,False,214,NaN
0,450958,False,48.187.667,1948,False,,,[],None,Islamic Art,...,,,2021-11-29T10:04:10.363Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",,False,,NaN
0,751497,True,2017.717,2017,True,https://images.metmuseum.org/CRDImages/ad/orig...,https://images.metmuseum.org/CRDImages/ad/web-...,[],None,The American Wing,...,,,2022-06-02T12:28:57.13Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,https://www.wikidata.org/wiki/Q104413096,False,,NaN
0,85512,False,1975.344.4,1975,False,,,[],None,Islamic Art,...,,,2020-12-30T04:37:58.547Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",,False,,NaN
0,227775,False,54.203,1954,False,,,[],None,European Sculpture and Decorative Arts,...,,,2022-02-09T04:42:08.947Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",,False,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,352478,False,38.52,1938,True,https://images.metmuseum.org/CRDImages/dp/orig...,https://images.metmuseum.org/CRDImages/dp/web-...,[https://images.metmuseum.org/CRDImages/dp/ori...,"[{'constituentID': 83628, 'role': 'Editor', 'n...",Drawings and Prints,...,,,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Men', 'AAT_URL': 'http://vocab.gett...",,True,,NaN
0,368861,False,25.62.32r(a),1925,False,,,[],"[{'constituentID': 166181, 'role': 'Artist', '...",Drawings and Prints,...,,,2021-08-20T04:36:13.79Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,,False,,NaN
0,361192,False,20.81.3.167,1920,False,,,[],"[{'constituentID': 212471, 'role': 'Etcher', '...",Drawings and Prints,...,,,2021-08-20T04:36:13.79Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,,False,,NaN
0,770912,False,18.17.1-45,1918,False,,,[],"[{'constituentID': 95757, 'role': 'Artist', 'n...",Drawings and Prints,...,,,2021-06-05T04:45:37.2Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,None,,False,,NaN


In [134]:
df_robe.to_csv(r'scrapped_data/MET_robe.csv', index = False)

### prepare to download image

In [102]:
df_robe = pd.read_csv('scrapped_data/MET_robe.csv')

In [103]:
df_robe.shape

(2470, 58)

In [104]:
condition1 = (df_robe['isPublicDomain'] == True) # select the ones is public domain
condition2 = (df_robe['primaryImage'] != '') # select the ones has primary images

df_sel = df_robe.loc[condition1 & condition2]

In [105]:
df_sel['classification'].unique()

array(['Costumes-Embroidered', nan, 'Main dress-Womenswear', 'Main dress',
       'Outerwear-Womenswear', 'Main dress-Menswear',
       'Night and Dressing Wear', 'Costumes', 'Textiles-Costumes',
       'Costumes-Tapestries', 'Ceramics', 'Textiles-Embroidered',
       'Main dress-Childrenswear', 'Crèche', 'Paintings',
       'Costumes-Woven', 'Textiles-Woven', 'Textiles-Velvets',
       'Sculpture', 'Hide-Costumes', 'Codices', 'Textiles-Laces',
       'Glass-Stained', 'Prints', 'Photographs', 'Textiles-Tapestries',
       'Drawings', 'Lacquer', 'Furniture', 'Costumes-Velvets', 'Textiles',
       'Metalwork-Lead', 'Textiles-Painted and Dyed',
       'Textiles-Painted and Printed', 'Calligraphy',
       'Textiles-Dyed and Embroidered', 'Outerwear-Menswear', 'Netsuke',
       'Illustrated Books', 'Lapidary Work-Sardonyx', 'Metalwork-Silver',
       'Metalwork', 'Jade', 'Enamels-Basse taille', 'Inrō',
       'Textiles-Dyed', 'Idiophone-Shaken-crotal bell', 'Sculpture-Stone',
       'Gold a

In [106]:
sel = ['Costumes-Embroidered', 'Main dress-Womenswear', 'Main dress',
       'Outerwear-Womenswear', 'Main dress-Menswear',
       'Night and Dressing Wear', 'Costumes', 'Textiles-Costumes',
       'Costumes-Tapestries', 'Ceramics', 'Textiles-Embroidered',
       'Main dress-Childrenswear', 
       'Costumes-Woven', 'Costumes-Velvets', 'Outerwear-Menswear']

df_sel[df_sel['classification'].isin(sel)]

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message
0,53715.0,False,43.119,1943.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.get...",NaN,False,214.0,NaN
21,86241.0,False,C.I.69.5,1969.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2021-04-20T04:40:27.437Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
22,85550.0,False,C.I.54.15,1954.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2021-04-20T04:40:27.437Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
23,85503.0,False,1975.388.1,1975.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2021-04-06T04:41:04.967Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
28,126846.0,False,C.I.41.86.7,1941.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,[],NaN,Islamic Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,448246.0,False,"29.106a, b",1929.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2022-09-09T04:54:13.663Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Men', 'AAT_URL': 'http://vocab.gett...",NaN,False,NaN,NaN
2225,229059.0,False,64.101.1363,1964.0,True,https://images.metmuseum.org/CRDImages/es/orig...,https://images.metmuseum.org/CRDImages/es/web-...,['https://images.metmuseum.org/CRDImages/es/or...,NaN,European Sculpture and Decorative Arts,...,NaN,NaN,2021-06-29T04:39:38.347Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Faces', 'AAT_URL': 'http://vocab.ge...",NaN,True,NaN,NaN
2262,447958.0,False,27.13.9,1927.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,True,NaN,NaN
2284,452400.0,False,1972.120.3,1972.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,True,NaN,NaN


In [107]:
df_sel['department'].value_counts()

Asian Art                                 781
European Paintings                        159
Costume Institute                         149
Drawings and Prints                       126
Islamic Art                               121
European Sculpture and Decorative Arts     79
Ancient Near Eastern Art                   54
Medieval Art                               30
The Cloisters                              28
Photographs                                26
Robert Lehman Collection                   24
The Michael C. Rockefeller Wing            11
The American Wing                          11
Egyptian Art                               10
Arms and Armor                              8
Greek and Roman Art                         4
Musical Instruments                         3
Name: department, dtype: int64

In [108]:
df_sel['objectID'] = df_sel['objectID'].astype('int')

<ipython-input-108-46fe0f398f49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['objectID'] = df_sel['objectID'].astype('int')


In [109]:
df_sel[df_sel['objectName'].str.contains('robe')==True]

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message
2,751497,True,2017.717,2017.0,True,https://images.metmuseum.org/CRDImages/ad/orig...,https://images.metmuseum.org/CRDImages/ad/web-...,[],NaN,The American Wing,...,NaN,NaN,2022-06-02T12:28:57.13Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,https://www.wikidata.org/wiki/Q104413096,False,NaN,NaN
115,69855,False,46.187.6,1946.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Peonies', 'AAT_URL': 'http://vocab....",NaN,False,NaN,NaN
120,68577,False,30.75.98,1930.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2022-09-13T04:54:15.93Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,213.0,NaN
132,229782,False,1983.574.2a–c,1983.0,True,https://images.metmuseum.org/CRDImages/es/orig...,https://images.metmuseum.org/CRDImages/es/web-...,['https://images.metmuseum.org/CRDImages/es/or...,NaN,European Sculpture and Decorative Arts,...,NaN,NaN,2020-09-16T18:35:19.457Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
134,454043,False,2003.416a–e,2001.0,True,https://images.metmuseum.org/CRDImages/is/orig...,https://images.metmuseum.org/CRDImages/is/web-...,['https://images.metmuseum.org/CRDImages/is/or...,NaN,Islamic Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,True,460.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,70233,False,58.97.2,1958.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN,NaN
892,70397,False,65.217,1965.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-04-22T05:09:02.033Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Dragons', 'AAT_URL': 'http://vocab....",NaN,False,NaN,NaN
901,53619,False,45.123.4,1945.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Costumes', 'AAT_URL': 'http://vocab...",NaN,False,NaN,NaN
1028,61806,False,2001.428.23,2001.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Leaves', 'AAT_URL': 'http://vocab.g...",NaN,False,NaN,NaN


In [110]:
df_sel[df_sel['title'].str.contains('robe') == True]

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message
0,53715,False,43.119,1943.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.get...",NaN,False,214.0,NaN
2,751497,True,2017.717,2017.0,True,https://images.metmuseum.org/CRDImages/ad/orig...,https://images.metmuseum.org/CRDImages/ad/web-...,[],NaN,The American Wing,...,NaN,NaN,2022-06-02T12:28:57.13Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,https://www.wikidata.org/wiki/Q104413096,False,NaN,NaN
116,69900,False,49.32.125,1949.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Cranes', 'AAT_URL': 'http://vocab.g...",NaN,False,213.0,NaN
120,68577,False,30.75.98,1930.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2022-09-13T04:54:15.93Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,213.0,NaN
132,229782,False,1983.574.2a–c,1983.0,True,https://images.metmuseum.org/CRDImages/es/orig...,https://images.metmuseum.org/CRDImages/es/web-...,['https://images.metmuseum.org/CRDImages/es/or...,NaN,European Sculpture and Decorative Arts,...,NaN,NaN,2020-09-16T18:35:19.457Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Robes', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,55960,False,1972.18,1972.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,NaN,Asian Art,...,NaN,NaN,2021-12-01T04:35:59Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Books', 'AAT_URL': 'http://vocab.ge...",NaN,False,NaN,NaN
1348,816191,False,2018.853.4a–c,2018.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,"[{'constituentID': 37502, 'role': 'Artist', 'n...",Asian Art,...,NaN,NaN,2022-06-02T12:28:57.13Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,https://www.wikidata.org/wiki/Q78610277,False,NaN,NaN
1685,675966,False,2015.741.9,2015.0,True,https://images.metmuseum.org/CRDImages/es/orig...,https://images.metmuseum.org/CRDImages/es/web-...,['https://images.metmuseum.org/CRDImages/es/or...,NaN,European Sculpture and Decorative Arts,...,NaN,NaN,2022-05-20T05:08:55.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Men', 'AAT_URL': 'http://vocab.gett...",NaN,False,NaN,NaN
1686,675971,False,2015.741.14,2015.0,True,https://images.metmuseum.org/CRDImages/es/orig...,https://images.metmuseum.org/CRDImages/es/web-...,['https://images.metmuseum.org/CRDImages/es/or...,NaN,European Sculpture and Decorative Arts,...,NaN,NaN,2022-05-20T05:08:55.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Men', 'AAT_URL': 'http://vocab.gett...",NaN,False,NaN,NaN


In [111]:
df_sel = df_sel[df_sel['primaryImage'].isna() != True]
df_sel.shape

(1623, 58)

In [112]:
df_sel

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber,message
0,53715,False,43.119,1943.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,[],NaN,Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Bats', 'AAT_URL': 'http://vocab.get...",NaN,False,214.0,NaN
2,751497,True,2017.717,2017.0,True,https://images.metmuseum.org/CRDImages/ad/orig...,https://images.metmuseum.org/CRDImages/ad/web-...,[],NaN,The American Wing,...,NaN,NaN,2022-06-02T12:28:57.13Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,https://www.wikidata.org/wiki/Q104413096,False,NaN,NaN
7,108103,False,1975.227.9,1975.0,True,https://images.metmuseum.org/CRDImages/ci/orig...,https://images.metmuseum.org/CRDImages/ci/web-...,['https://images.metmuseum.org/CRDImages/ci/or...,NaN,Costume Institute,...,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN,NaN
8,102549,False,45.73,1945.0,True,https://images.metmuseum.org/CRDImages/ci/orig...,https://images.metmuseum.org/CRDImages/ci/web-...,[],NaN,Costume Institute,...,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN,NaN
9,107996,False,1972.78.3,1972.0,True,https://images.metmuseum.org/CRDImages/ci/orig...,https://images.metmuseum.org/CRDImages/ci/web-...,['https://images.metmuseum.org/CRDImages/ci/or...,NaN,Costume Institute,...,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,436797,False,41.190.19,1941.0,True,https://images.metmuseum.org/CRDImages/ep/orig...,https://images.metmuseum.org/CRDImages/ep/web-...,['https://images.metmuseum.org/CRDImages/ep/or...,"[{'constituentID': 162015, 'role': 'Artist', '...",European Paintings,...,NaN,NaN,2022-11-02T04:54:43.4Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Virgin Mary', 'AAT_URL': 'http://vo...",https://www.wikidata.org/wiki/Q19904858,False,NaN,NaN
2460,53233,False,2015.300.67,2015.0,True,https://images.metmuseum.org/CRDImages/as/orig...,https://images.metmuseum.org/CRDImages/as/web-...,['https://images.metmuseum.org/CRDImages/as/or...,"[{'constituentID': 11724, 'role': 'Artist', 'n...",Asian Art,...,NaN,NaN,2022-10-20T04:55:06.267Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Musical Instruments', 'AAT_URL': 'h...",https://www.wikidata.org/wiki/Q78760038,False,NaN,NaN
2461,435838,False,05.39.2,1905.0,True,https://images.metmuseum.org/CRDImages/ep/orig...,https://images.metmuseum.org/CRDImages/ep/web-...,['https://images.metmuseum.org/CRDImages/ep/or...,"[{'constituentID': 161740, 'role': 'Artist', '...",European Paintings,...,NaN,NaN,2022-10-31T04:54:57.077Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Interiors', 'AAT_URL': 'http://voca...",https://www.wikidata.org/wiki/Q19911530,False,NaN,NaN
2462,435640,False,30.95.256,1930.0,True,https://images.metmuseum.org/CRDImages/ep/orig...,https://images.metmuseum.org/CRDImages/ep/web-...,[],"[{'constituentID': 161668, 'role': 'Artist', '...",European Paintings,...,NaN,NaN,2022-10-31T04:54:57.077Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Madonna and Child', 'AAT_URL': 'htt...",https://www.wikidata.org/wiki/Q

### download 

In [113]:
df_sel.shape

(1623, 58)

In [236]:
download_img(df_sel,'robe/')

start!
error id 45088
1 errors in total
error id 69010
2 errors in total
error id 85436
3 errors in total
error id 68295
4 errors in total
error id 86895
5 errors in total
error id 81687
6 errors in total
error id 84299
7 errors in total
error id 86178
8 errors in total
error id 38123
9 errors in total
error id 437028
10 errors in total
error id 61833
11 errors in total
error id 468576
12 errors in total
error id 82630
13 errors in total
error id 105411
14 errors in total
error id 84436
15 errors in total
error id 84414
16 errors in total
error id 82158
17 errors in total
error id 436908
18 errors in total
3763.592482328415 secs slipped...


## Scrape Asian Art + keywords

### download dataframe

In [3]:
df_asian = pd.read_csv('tran_file\df_asian.csv') # this csv generated in notebook 'data wrangling'
df_asian.shape

(682, 54)

In [101]:
idlist = df_asian['Object ID'].to_list()
print(f'{len(idlist)} objects')

682 objects


In [13]:
df_asian_api = scrapping_by_idlist(idlist)

111.81104588508606 secs slipped...


In [20]:
df_asian_api.to_csv('scrapped_data/MET_Asian_Art_Keywords.csv',index=False)

In [94]:
df_asian_api = pd.read_csv('scrapped_data/MET_Asian_Art_Keywords.csv')

### download images

In [96]:
df_asian_api = df_asian_api[df_asian_api['primaryImage'] != '']

In [97]:
df_asian_api.shape

(471, 57)

In [18]:
download_img(df_asian_api,'asian_art/')

start!
error id 45088
1 errors in total
error id 68295
2 errors in total
error id 69010
3 errors in total
error id 70630
4 errors in total
error id 88642
5 errors in total
error id 97475
6 errors in total
error id 126433
7 errors in total
598.0307347774506 secs slipped...


## scrape Full Selection

In [2]:
df_full_selection = pd.read_csv('tran_file/df_full_selection.csv')

In [3]:
df_full_selection.shape

(14292, 54)

In [4]:
idlist = df_full_selection['Object ID'].to_list()
print(f'{len(idlist)} objects')

14292 objects


### download dataframe

In [10]:
df_full_selection_api = scrapping_by_idlist(idlist)

2100.1011135578156 secs slipped...


In [11]:
df_full_selection_api.to_csv('scrapped_data/MET_full_selection_Keywords.csv',index=False)

In [12]:
df_full_selection_api = pd.read_csv('scrapped_data/MET_full_selection_Keywords.csv')

### download images

In [13]:
df_full_selection_api = df_full_selection_api[df_full_selection_api['primaryImage'] != '']

In [14]:
df_full_selection_api.shape

(14292, 57)

In [19]:
df_full_selection_api[df_full_selection_api['isPublicDomain']==False]

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,...,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,78944,False,"1996.61.3a, b",1996.0,False,NaN,NaN,[],"[{'constituentID': 21169, 'role': 'Design Hous...",Costume Institute,...,NaN,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
1,78945,False,"1996.61.4a, b",1996.0,False,NaN,NaN,[],"[{'constituentID': 21169, 'role': 'Design Hous...",Costume Institute,...,NaN,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
2,78946,False,1996.61.5a–c,1996.0,False,NaN,NaN,[],"[{'constituentID': 21169, 'role': 'Design Hous...",Costume Institute,...,NaN,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
3,78947,False,"1996.61.6a, b",1996.0,False,NaN,NaN,[],"[{'constituentID': 21169, 'role': 'Design Hous...",Costume Institute,...,NaN,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
4,78949,False,"1996.61.8a, b",1996.0,False,NaN,NaN,[],"[{'constituentID': 25847, 'role': 'Design Hous...",Costume Institute,...,NaN,NaN,NaN,2021-06-05T04:45:37.2Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14286,75954,False,"2011.244a, b",2011.0,False,NaN,NaN,[],NaN,Asian Art,...,Textiles-Costumes,NaN,NaN,2020-04-02T14:22:10.83Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
14287,85785,False,1977.90,1977.0,False,NaN,NaN,[],NaN,Asian Art,...,Main dress,NaN,NaN,2021-04-14T04:40:17.717Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN
14288,86210,False,C.I.41.124.46,1941.0,False,NaN,NaN,[],NaN,Asian Art,...,Main dress-Womenswear,NaN,NaN,2021-04-14T04:40:17.717Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,"[{'term': 'Animals', 'AAT_URL': 'http://vocab....",NaN,False,NaN
14290,86227,False,1980.383.1,1980.0,False,NaN,NaN,[],NaN,Asian Art,...,Main dress-Womenswear,NaN,NaN,2021-04-14T04:40:17.717Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,NaN,NaN,False,NaN


In [15]:
download_img(df_full_selection_api,'full_selection/')

14292 images will be downloaded.
error id 78944
1 errors in total
error id 78945
2 errors in total
error id 78946
3 errors in total
error id 78947
4 errors in total
error id 78949
5 errors in total
error id 78951
6 errors in total
error id 78964
7 errors in total
error id 78974
8 errors in total
error id 78975
9 errors in total
error id 78987
10 errors in total
error id 79001
11 errors in total
error id 79003
12 errors in total
error id 79004
13 errors in total
error id 79053
14 errors in total
error id 79113
15 errors in total
error id 79144
16 errors in total
error id 79179
17 errors in total
error id 79181
18 errors in total
error id 79206
19 errors in total
error id 79257
20 errors in total
error id 79261
21 errors in total
error id 79277
22 errors in total
error id 79296
23 errors in total
error id 79302
24 errors in total
error id 79303
25 errors in total
error id 79304
26 errors in total
error id 79309
27 errors in total
error id 79310
28 errors in total
error id 79323
29 errors

## (not done) beautiful soup

In [12]:
df_full_selection_api = pd.read_csv('scrapped_data\MET_full_selection_Keywords.csv')
df_full_selection_api.head(1)

,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,objectName,title,culture,period,dynasty,reign,portfolio,artistRole,artistPrefix,artistDisplayName,artistDisplayBio,artistSuffix,artistAlphaSort,artistNationality,artistBeginDate,artistEndDate,artistGender,artistWikidata_URL,artistULAN_URL,objectDate,objectBeginDate,objectEndDate,medium,dimensions,measurements,creditLine,geographyType,city,state,county,country,region,subregion,locale,locus,excavation,river,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,78944,False,"1996.61.3a, b",1996.0,False,NaN,NaN,[],"[{'constituentID': 21169, 'role': 'Design House', 'name': 'Perry Ellis Sportswear Inc.', 'constituentULAN_URL': '', 'constituentWikidata_URL': '', 'gender': ''}, {'constituentID': 20148, 'role': 'Designer', 'name': 'Perry Ellis', 'constituentULAN_URL': '', 'constituentWikidata_URL': '', 'gender': ''}]",Costume Institute,Suit,Suit,American,NaN,NaN,NaN,NaN,Design House,NaN,Perry Ellis Sportswear Inc.,"American, founded 1978",NaN,Ellis Perry Sportswear Inc.,American,1978,9999,NaN,NaN,NaN,ca. 1980,1975,1985,cotton,NaN,"[{'elementName': 'Length at CB', 'elementDescription': 'a', 'elementMeasurements': {'Length': 53.3401}}, {'elementName': 'Length at CB', 'elementDescription': 'b', 'elementMeasurements': {'Length': 76.2002}}]","Gift of Roz Gerten Jacobs, 1996",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-02T21:50:01.377Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/78944,NaN,NaN,False,NaN


In [15]:
df_full_selection_api.loc[df_full_selection_api['objectID']==78944,'objectURL']

0    https://www.metmuseum.org/art/collection/search/78944
Name: objectURL, dtype: object

In [41]:
costume_url = "https://www.metmuseum.org/art/collection/search/78944"

costume_req = requests.get(costume_url)
costume_soup = BeautifulSoup(costume_req.text,'html.parser')

In [42]:
costume_soup

<html>
<head>
<meta content="noindex,nofollow" name="robots"/>
<script src="/_Incapsula_Resource?SWJIYLWA=5074a744e2e3d891814e9a2dace20bd4,719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<body>
</body></head></html>

In [49]:
costume_fact = costume_soup.find_all('img')

costume_fact

[]

In [56]:
ardbeg_url = "https://www.dpm.org.cn/court/culture/156240.html"

ardbeg_req = requests.get(ardbeg_url)
ardbeg_soup = BeautifulSoup(ardbeg_req.content)


In [57]:
ardbeg_soup.find_all('img')

[<img alt="故宫博物院" class="red" src="https://img.dpm.org.cn/Uploads/Picture/2021/08/19/s611e293c9d212.png"/>,
 <img alt="故宫博物院" class="wht" src="https://img.dpm.org.cn/Uploads/Picture/2021/08/19/s611e2933ab432.png"/>,
 <img alt="故宫博物院" class="red" src="https://img.dpm.org.cn/Uploads/Picture/2021/08/19/s611e2946b7886.png"/>,
 <img src="https://img.dpm.org.cn/Public/static/themes/image/sidebar_gotop1.png"/>,
 <img id="noheart" src="https://img.dpm.org.cn/Public/static/themes/image/sidebar_bg2.png"/>,
 <img id="heart" src="https://img.dpm.org.cn/Public/static/themes/image/sidebar_bg2-1.png" style="display:none;"/>,
 <img src="https://img.dpm.org.cn/Public/static/themes/image/sidebar_bg1.png"/>,
 <img data-cmd="tsina" src="https://img.dpm.org.cn/Public/static/themes/image/lyc/share-icon1.png"/>,
 <img data-cmd="weixin" src="https://img.dpm.org.cn/Public/static/themes/image/lyc/share-icon2.png"/>,
 <img src="https://img.dpm.org.cn/Public/static/themes/image/icon_mess1.png"/>,
 <img src="https